# IBA Probe for MLAG Traffic Imbalance

In this tutorial we will demonstrate the "MLAG Traffic Imbalance" probe that is used to messure traffic load on your MLAG interfaces and detect imbalances between links and devices in the pair.

The first thing we need to do to is setup our AOS server and identify the blueprint we want to use.

In [1]:
aos_server = '172.20.91.3'
blueprint_name = "rack-based-blueprint-f03bfadb"

Next let's create a AOS python client that we will use to create our probe

In [2]:
import json
from aospy.swagger import Client

aos = Client('https://' + aos_server)

Now let's use that client to get the blueprint, so we can have the blueprint_id value we will need when we post the probe.

In [3]:
from aospy.swagger.groups.blueprints import Blueprints
bp_index = Blueprints(aos)
bp = bp_index[blueprint_name]

In [4]:
print bp.id

rack-based-blueprint-f03bfadb


Now let's import the MLAG imbalance probe from the library

In [5]:
from aospy.ibaprobelib.mlag_imbalance import mlag_imbalance_probe

Next let's create the MLAG imbalance probe payload information that we will need.  The probe takes a few parmateres which we can see if we introspect the function:

In [ ]:
?mlag_imbalance_probe

The following paramaters are required:

`label` - Descriptive name for the probe   
`duration` - number of seconds of recent-history in which we will consider ECMP imbalance  
`std_max` - maxiumum standard deviation used for imbalance detection   

So for the case of reuse, we will define these variables here:

In [6]:
probe_label = 'DC1-MLAG-Imbalance'
duration = 120
std_max = 2

Now let's create the body data from these variables:

In [7]:
probe_body = mlag_imbalance_probe(label=probe_label, duration=duration, std_max=std_max)

If you'd like to dump the contents of the probe_body, you can use the following:

In [8]:
# print json.dumps(probe_body, indent=2)

Finally, post the probe body to AOS to create the probe.

In [9]:
probes_url = bp.url + "/probes"

In [10]:
probes_url

u'https://172.20.91.3/api/blueprints/rack-based-blueprint-f03bfadb/probes'

In [11]:
resp = aos.remote.api.post(probes_url, json=probe_body)

In [12]:
resp.ok

True

Now we have our IBA Probe ID:

In [13]:
probe_id = resp.json()['id']

print probe_id

5f968fa9-4146-4996-9291-cb64d51b3e38


And now we can see the IBA active probe in the UI:

<img src="media/mlag_imbalance1.png">

And then select the probe on the UI to get the details:

<img src="media/mlag_imbalance2.png">

# Delete the IBA MLAG Traffic Imbalance Probe

If you want to remove the IBA probe, you can use the API to do so, like the following; just uncomment this out.

In [14]:
resp = aos.remote.api.delete(probes_url + "/%s" % probe_id)
resp.ok

True